In [1]:
ls -l ../../

total 8
-rw-r--r--   1 mabu-dev  staff  104 Oct  7 19:12 README.md
drwxr-xr-x  15 mabu-dev  staff  480 Oct  7 19:14 qaphelaMobile/
drwxr-xr-x   8 mabu-dev  staff  256 Oct 11 23:01 qaphelaWeb/


In [2]:
import pandas as pd
import numpy as np
from datetime import timedelta, datetime
import sys,json
import os
import sqlalchemy as sa
sys.path.append("../")
import django
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'qaphelaWeb.settings')
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
django.setup()
from api import models
from django.db.models import Count, Avg, Q, Prefetch, F, Subquery

from IPython.display import clear_output

In [3]:
scr_db = {
    'NAME': 'deephealthdb',
    'USER': 'deephealth',
    'PASSWORD': 'd33ph3a1th',
    # 'HOST': '35.228.3.60',
    'HOST': 'localhost',
    'PORT': '5432'
}

In [4]:
db = scr_db
engine_string = "postgresql+psycopg2://{user}:{password}@{host}:{port}/{database}".format(
    user = db['USER'],
    password = db['PASSWORD'],
    host = db['HOST'],
    port = db['PORT'],
    database = db['NAME'],
)

engine = sa.create_engine(engine_string)
engine.connect()

In [5]:
address = pd.read_sql_table('api_address', engine)
address.head()

,id,address,city,country,lattitude,longitude,postal_code,province,street_name,street_number,suburb,graph_id
0,1617,"Farrarmere, Benoni, 1501, South Africa",Benoni,South Africa,-26.1681312,28.302357899999997,1501.0,Gauteng,nan,nan,Farrarmere,394
1,1759,"Florida, USA",nan,nan,27.664827399999997,-81.5157535,nan,nan,nan,nan,nan,643
2,3598,"1 Chief Albert Luthuli Dr, Potchefstroom, 2531...",Potchefstroom,South Africa,-26.700290399999997,27.096456300000003,2531,North West,Chief Albert Luthuli Drive,1,Potchefstroom,1176
3,3518,"46 Salazar Plain, Cape Town City Centre, Cape ...",Cape Town,South Africa,-33.9184668,18.4296845,8000,Western Cape,Salazar Plain,46,nan,1507
4,3163,"130 Bram Fischer Dr, Ferndale, Randburg, 2194,...",Randburg,South Africa,-26.10462,28.00252,2194,Gauteng,Bram Fischer Drive,130,Ferndale,1610


In [6]:
address = address.sample(50)
address.shape

(50, 12)

In [7]:
# contact_numbers = models.ContactDetails.objects.create(
#     primary_contact_number = '0817189287',
#     secondary_contact_number = '0798595080',
#     next_of_kin_firstname = 'Mabu',
#     next_of_kin_lasttname = 'Manaileng',
#     next_of_kin_primary_contact_number = '0817189287')
contact_numbers = models.ContactDetails.objects.get(id=1)
contact_numbers

<ContactDetails: ContactDetails object (1)>

In [8]:
cols = list(address.columns)
cols.remove("id")
cols.remove("graph_id")
cols

['address',
 'city',
 'country',
 'lattitude',
 'longitude',
 'postal_code',
 'province',
 'street_name',
 'street_number',
 'suburb']

In [9]:
# for row in address.values:
#     row = list(row)
#     row = {col:val for col, val in zip(cols, row[1:-1])}
#     addr = models.Address.objects.create(
#         address = row['address'],
#         city = row['city'],
#         country = row['country'],
#         lattitude = row['lattitude'],
#         longitude = row['longitude'],
#         postal_code = row['postal_code'],
#         province = row['province'],
#         street_name = row['street_name'],
#         street_number = row['street_number'],
#         suburb = row['suburb'])
#     print(addr)

In [20]:
contact_numbers = models.ContactDetails.objects.create(
    primary_contact_number = '0798595080')
contact_numbers = models.ContactDetails.objects.get(id=2)
contact_numbers

<ContactDetails: ContactDetails object (2)>

In [11]:
# social_profile = models.SocialProfiles.objects.create(
#     name="Twitter",
#     profile_page="https://twitter.com/thisMabu"
# )

social_profile = models.SocialProfiles.objects.get(id=1)

<SocialProfiles: SocialProfiles object (1)>

In [40]:
from django.contrib.auth.hashers import make_password

In [15]:
# user = django.contrib.auth.models.User.objects.create(
#     username="0817189287",
#     first_name="Mabu",
#     last_name="Manaileng",
#     is_active=True,
#     email="ManailengMJ@gmail.com",
#     password=make_password("11235")
# )
user = django.contrib.auth.models.User.objects.get(id=1)
user

<User: 0817189287>

In [48]:
# models.QaphelaUser.objects.all().delete()

In [16]:
# quser = models.QaphelaUser.objects.create(
#     user = user,
#     user_type = "STANDARD",
#     is_dispatch_user = True,
#     social_profiles = social_profile,
#     contact_details = contact_numbers,
#     address = addr)
quser = models.QaphelaUser.objects.get(user_id=1)
quser

<QaphelaUser: QaphelaUser object (1)>

In [21]:
models.Abuser.objects.all().delete()

(2, {'api.Case': 1, 'api.Abuser': 1})

In [24]:
# abuser = models.Abuser.objects.create(
#     full_names = "Diego Masemola",
#     social_profiles = social_profile,
#     contact_details = contact_numbers,
#     workplace_details = "Research Scientist at Datawizzards as of October 2020")
abuser = models.Abuser.objects.get(id=2)
abuser

<Abuser: Abuser object (2)>

In [26]:
models.Case.objects.all() #.delete()

<QuerySet []>

In [27]:
case = models.Case.objects.create(
    title = "Diego Masemola raped me",
    report_type = "REPORT",
    occasion = "WORK",
    preview = "We were at a work function when we both got drunk. \
        He forced to come to my room with me and forced himself on me, after I repeatedly said NO!, STOP!\
            I reported him to HR and he admitted guilt, he was subsequently fired from the company.\
                 We settled things out of court.",
    when = "2020-02-23",
    perpetrators = abuser,
    reporter = quser,
    frequency = 1,
    case_status = "VERIFIED",
    settlement_type = "OUT OF COURT",
    police_case_number = "2020/01/234")
case

<Case: Case object (2)>